# High Performance Seismology — Wavefield Simulations Using SPECFEM
## Notebook 3: Intro to SPECFEM3D

- SPECFEM3D follows very similar practices as SPECFEM2D.  
- What we have learned thus far will help us understand how to run simulations in SPECFEM3D.  
- In this notebook we will be using SPECFEM3D_Cartesian.  
- **Any subsequent use of the term 'SPECFEM3D' will refer to  'SPECFEM3D_cartesian'** and not the other variants (SPECFEM3D_GLOBE, etc).
- The best reference on how to set up and use SPECFEM3D would be the user [manual](https://github.com/geodynamics/specfem3d/blob/devel/doc/USER_MANUAL/manual_SPECFEM3D_Cartesian.pdf).  
- SPECFEM3D has already been set up in our container, and here we will focus on how to use it.   
- In this short introduction, we will explore the SPECFEM3D repository, and then run small example problems with 3D models, starting with a homogeneous halfspace example. 
- For simplicity we will restrict the 3D models to essentially 1D models in this workshop.  
- These instructions should be run from inside the Docker container, using Jupyter Lab (see instructions [here](https://github.com/adjtomo/adjdocs/blob/main/readmes/docker_image_install.md)).

-----------

**Relevant Links:** 
- Today's Notebook: https://github.com/adjtomo/adjdocs/blob/main/workshops/2023-05-12_hps/3_intro_specfem3d.ipynb  
- SPECFEM3D_Cartesian User Manual: https://github.com/geodynamics/specfem3d/blob/devel/doc/USER_MANUAL/manual_SPECFEM3D_Cartesian.pdf
- SPECFEM3D_Cartesian GitHub Repository: https://github.com/geodynamics/specfem3d/tree/devel  

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.
- To time a task, put a `%time` before the command (e.g., `%time ! ls`)

------------
## 1) Package Exploration

Let's get started by looking at the SPECFEM3D repository.

In [ ]:
# Python packages we will need for this notebook
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image

In [ ]:
# Go to the SPECFEM3D directory
%cd /home/scoped/specfem3d

# Exploring the SPECFEM3D directory
! ls

Among the many folders (and files) listed above, the most notable ones are -
- `bin/`
- `DATA/`
- `OUTPUT_FILES/`
- `EXAMPLES/`
- `src/`

Let's look at these folders one by one.

### 1a) ' bin/ ' directory

In [ ]:
! ls bin

- The `bin/` folder contains binary executable files which are essentially linked compiled fortran code generated after compiling various programs available with this package  
- The most essential executables for a basic simulation are:  
    - `xmeshfem3D`: also called SPECFEM3D's internal mesher, this program discretizes the simulation domain into small elements  
    - `xgenerate_databases`: this database generation program assigns model parameter values to the elements  
    - `xspecfem3D`: also called the solver, this program uses the mesh and the corresponding model parameter values assigned to the mesh elements, for numerically solving the wave equation  
    
### 1b) ' DATA/ ' directory


In [ ]:
! ls DATA

The `DATA/` folder is the input files folder and contains files and folders which describe the -
- mesh: (`meshfem3D_files/`)
- model: (`meshfem3D_files/`, `tomo_files/`)
- source: (`CMTSOLUTION`, `FORCESOLUTION`)
- station: (`STATIONS`)

### 1c) ' OUTPUT_FILES/ ' directory


In [ ]:
! ls OUTPUT_FILES

- This `OUTPUT_FILES/` folder contains the output files of any SPECFEM3D job.   
- The `DATABASES_MPI/` folder in the `OUTPUT_FILES/` folder contains the database files generated as a result of a meshing or database generation job. The database files can be very large in size.  
- Other job files like log files for the various programs as well as the output seismograms are directly generated in the `OUTPUT_FILES/` folder.

### 1d) ' src/ ' directory

In [ ]:
! ls src

- The `src/` folder contains the source code for the various programs in SPECFEM3D.  
- If you want to add to or modify some of SPECFEM3D's features, you need to modify one or more of the files in the 'src/' subfolders. 
- If you think the features you added would be useful to the broader SPECFEM3D community, please consider making a pull request to the SPECFEM3D github [repository](https://github.com/geodynamics/specfem3d/tree/devel) so that it can be reviewed and integrated to the package.  

### 1e) ' EXAMPLES/ ' directory

In [ ]:
! ls EXAMPLES

- The `EXAMPLES/` folder contains a variety of examples to provide a quick start to the user on how to use SPECFEM3D and its various features. 
- The README files within the example folders within `EXAMPLES/` guide the user through the steps to run each example.  
- Some of the highly recommended examples to start with, depending on your need to run SPECFEM3D, are:
    - `homogeneous_halfspace/` - a simple homogeneous halfspace model based simulation
    - `meshfem3D_examples/socal1D/` - a layered model based simulation
    - `sensitivity_kernels_liutromp2006/` - adjoint simulations to generate the kernels in [Liu&Tromp 2006](https://pubs.geoscienceworld.org/ssa/bssa/article/96/6/2383/146674/Finite-Frequency-Kernels-Based-on-Adjoint-Methods)

>__NOTE:__ README files, with steps to run these examples, should not undermine the detailing provided in the [manual](https://github.com/geodynamics/specfem3d/blob/master/doc/USER_MANUAL) on how to run SPECFEM3D.



- Here we will start with the `homogeneous halfspace/` example, and then move on to running simulations with layered models.  
- We will also discuss some strategies to design meshes using SPECFEM3D's internal mesher.  

-----------

## 2) Setting Up Simulations

- It is often desirable to run SPECFEM outside of the cloned SPECFEM repository, in order to keep files and outputs manageable. 
- SPECFEM3D only requires the following 3 directories for a successful run -
    - `bin/` (with compiled executables)
    - `DATA/` (with the necessary input files)
    - `OUTPUT_FILES/`

In this section we will set up a separate SPECFEM3D working directory to work with.

>__NOTE:__ The following cells assume that we are in the directory `/home/scoped/work/day_1/specfem3d_workdir`, so we must evaluate the '%cd' command (if needed) to ensure that cells work as expected.

In [ ]:
# Create separate working directory for SPECFEM3D
! mkdir -p /home/scoped/work/day_1/specfem3d_workdir

# Go to the SPECFEM3D working directory
%cd /home/scoped/work/day_1/specfem3d_workdir

# Symlink the binary files, and copy the relevant DATA/ directory
! ln -s /home/scoped/specfem3d/bin .
! mkdir -p OUTPUT_FILES/DATABASES_MPI

# Look at the work directory
! ls

- The work directory now has the `bin/` and the `OUTPUT_FILES/` folders.
- The `DATA/` folder will be added in the subsequent sections depending on the example we want to run.

-----------

## 3) Simulations

A SPECFEM3D simulation primarily involves three steps -
- meshing
- database generation
- solving the wave equation

We will go through these steps in the following example.

### 3a) Homogeneous Halfspace

This example creates a homogeneous halfspace, i.e. a single volume block with a
constant elastic material property, using SPECFEM3D's internal mesher, and runs a
forward simulation.

In [ ]:
# Copy necessary input files for the homogeneous halfspace example
! cp -r /home/scoped/specfem3d/EXAMPLES/homogeneous_halfspace/DATA .
! cp -r /home/scoped/specfem3d/EXAMPLES/homogeneous_halfspace/meshfem3D_files DATA/.

# Look at the DATA directory
! ls DATA

#### Step 1) Meshing: Simulation domain discretization

In [ ]:
# Explore the mesh files
! ls DATA/meshfem3D_files

In [ ]:
# Check the Mesh_Par_file
! cat DATA/meshfem3D_files/Mesh_Par_file

In [ ]:
# Run mesher (xmeshfem3D)
! mpiexec -np 4 ./bin/xmeshfem3D

# Look at the generated mesh files
! ls OUTPUT_FILES/DATABASES_MPI

#### Step 2) Database Generation: Model assignment to mesh

In [ ]:
# Run database generator (xgenerate_databases)
! mpiexec -np 4 ./bin/xgenerate_databases

# Explore the generated database files
! ls OUTPUT_FILES/DATABASES_MPI

In [ ]:
# View the meshing and the Vp model
# The following image was generated using Paraview
! cp /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/figures/mesh/example_A.png .
Image("example_A.png")

**Fig.** The homogeneous halfspace velocity (Vp) model and mesh. The model spans 134 km in the 'X' and 'Y' directions, and 60 km in the 'Z' direction. The mesh has 36 elements in the 'X' and 'Y' directions, and has 16 elements in the 'Z' direction. The halfspace has a Vp value of 2.8 km/s.

#### Step 3) Simulation: Solving the wave equation

In [ ]:
# Check the source definition
! cat DATA/CMTSOLUTION

Read more about the CMTSOLUTION files [here](https://www.sciencedirect.com/science/article/pii/S0031920112000696?via%3Dihub).

In [ ]:
# Check the stations file
! cat DATA/STATIONS

In [ ]:
# View the meshing and the Vp model (figure pre-generated using ParaView)
! cp /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/figures/source_station_geometry/sr.png .
Image("sr.png")

**Fig.** The source station geometry, in the homogeneous halfspace domain with the 'Y' dimension cut in half. The source and stations are placed on the plane of the slice. The source is at the center of the grid, while the stations are equally spaced on a section along the surface.

In [ ]:
# Check the Par_file
! head -30 DATA/Par_file

In [ ]:
! head -80 DATA/Par_file | tail -n 22

In [ ]:
# Run the solver (xspecfem3D)
%time ! mpiexec -np 4 ./bin/xspecfem3D

>__NOTE:__ solver runs should take ~5 minutes or less on a recent (as of 2022) laptop.

Note the wall time for the simulation.

In [ ]:
# Look at the simulation output files
! ls OUTPUT_FILES/

In [ ]:
# Explore the seismograms
! head -25 OUTPUT_FILES/DB.X50.BXZ.semd 

In [ ]:
# Plot the seismograms
X20_Z = np.genfromtxt("OUTPUT_FILES/DB.X20.BXZ.semd", dtype=None, names=("time","BXZ"))
X30_Z = np.genfromtxt("OUTPUT_FILES/DB.X30.BXZ.semd", dtype=None, names=("time","BXZ"))
X40_Z = np.genfromtxt("OUTPUT_FILES/DB.X40.BXZ.semd", dtype=None, names=("time","BXZ"))
X50_Z = np.genfromtxt("OUTPUT_FILES/DB.X50.BXZ.semd", dtype=None, names=("time","BXZ"))

t = X20_Z["time"]

plt.title("Seismograms (Z - component)")
plt.xlabel("---- time -->")
plt.ylabel("---- displacement -->")

plt.plot(t,X20_Z["BXZ"],label="X20")
plt.plot(t,X30_Z["BXZ"],label="X30")
plt.plot(t,X40_Z["BXZ"],label="X40")
plt.plot(t,X50_Z["BXZ"],label="X50")

plt.legend(title="Station")
plt.savefig("seis.png")

In [ ]:
# Remove the large sized database files
! rm -rf /home/scoped/work/day_1/specfem3d_workdir/OUTPUT_FILES/DATABASES_MPI

# Archive the output files, and the mesh files
! mv sr.png OUTPUT_FILES/.
! mv example_A.png OUTPUT_FILES/mesh_examples_A.png
! mv seis.png OUTPUT_FILES/.
! mv OUTPUT_FILES OUTPUT_FILES_example_A
! mv DATA/meshfem3D_files DATA/meshfem3D_files_example_A

-----------

### 3b) Two Layered Model

This example is built upon the homogeneous halfspace model of the previous example, with the bottom three-fourths of the domain assigned higher velocities and density.

In [ ]:
# Setup
! cp -r /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/meshfem3D_files/example_B DATA/meshfem3D_files
! mkdir -p OUTPUT_FILES/DATABASES_MPI

! ls DATA/meshfem3D_files

In [ ]:
# Check the difference
! diff --color DATA/meshfem3D_files_example_A/Mesh_Par_file DATA/meshfem3D_files/Mesh_Par_file

- Here we assign higher velocities and density, to the bottom 45 km of the model, compared to that of the previous example, hence creating two separate layers.
- The domain discretization (meshing) remains the same.

>__NOTE:__ Here, the counting of various quantities, starts from the bottom layer

In [ ]:
# Run mesher (xmeshfem3D)
! mpiexec -np 4 ./bin/xmeshfem3D

# Run database generator (xgenerate_databases)
! mpiexec -np 4 ./bin/xgenerate_databases

In [ ]:
# View the meshing and the Vp model (figure pre-generated using ParaView)
! cp /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/figures/mesh/example_B.png .
Image("example_B.png")

**Fig.** The two layered velocity (Vp) model and mesh. The 15 km thick top layer has Vp = 2.8 km/s, while the 45 km thick bottom layer has Vp = 4.0 km/s. The discretization is the same as that used in the homogeneous halfspace example, with the top layer having 4 elements in the 'Z' direction, and the bottom layer having 12 elements in the 'Z' direction. 

In [ ]:
# Run the solver (xspecfem3D)
%time ! mpiexec -np 4 ./bin/xspecfem3D

>__NOTE:__ solver runs should take ~5 minutes or less on a recent (as on 2022) laptop.

Note the wall time for the simulation.

In [ ]:
# Plot the output seismograms
X20_Z = np.genfromtxt("OUTPUT_FILES/DB.X20.BXZ.semd", dtype=None, names=("time","BXZ"))
X30_Z = np.genfromtxt("OUTPUT_FILES/DB.X30.BXZ.semd", dtype=None, names=("time","BXZ"))
X40_Z = np.genfromtxt("OUTPUT_FILES/DB.X40.BXZ.semd", dtype=None, names=("time","BXZ"))
X50_Z = np.genfromtxt("OUTPUT_FILES/DB.X50.BXZ.semd", dtype=None, names=("time","BXZ"))

t = X20_Z["time"]

plt.title("Seismograms (Z - component)")
plt.xlabel("---- time -->")
plt.ylabel("---- displacement -->")

plt.plot(t,X20_Z["BXZ"],label="X20")
plt.plot(t,X30_Z["BXZ"],label="X30")
plt.plot(t,X40_Z["BXZ"],label="X40")
plt.plot(t,X50_Z["BXZ"],label="X50")

plt.legend(title="Station")
plt.savefig("seis.png")

In [ ]:
# Remove the large sized database files
! rm -rf /home/scoped/work/day_1/specfem3d_workdir/OUTPUT_FILES/DATABASES_MPI

# Archive the output files, and the mesh files
! mv example_B.png OUTPUT_FILES/mesh_examples_B.png
! mv seis.png OUTPUT_FILES/.
! mv OUTPUT_FILES OUTPUT_FILES_example_B
! mv DATA/meshfem3D_files DATA/meshfem3D_files_example_B

-----------

### 3c) Two Layered Model with Modified Mesh

This example is built upon the two layered model used in the previous example, with larger elements for the higher velocity layer.

In [ ]:
# Setup
! cp -r /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/meshfem3D_files/example_C DATA/meshfem3D_files
! mkdir -p OUTPUT_FILES/DATABASES_MPI

! ls DATA/meshfem3D_files

In [ ]:
# Check the difference
! diff --color DATA/meshfem3D_files_example_B/Mesh_Par_file DATA/meshfem3D_files/Mesh_Par_file 

- Here we have doubled the vertical dimension of the bottom layer elements compared to that of the previous example.
- We have also added a doubling layer to transition from smaller elements in the top layer to larger (double sized) elements in the bottom layer.
- Since the mesh is no longer regular, and has doubling layers, the number of elements along the 'X' and 'Y' direction at the surface need to be multiples of 8.

>__NOTE:__ Here, the counting of various quantities, starts from the bottom layer  

In [ ]:
# Run mesher (xmeshfem3D)
! mpiexec -np 4 ./bin/xmeshfem3D

# Run database generator (xgenerate_databases)
! mpiexec -np 4 ./bin/xgenerate_databases

In [ ]:
# View the meshing and the Vp model (figure pre-generated using ParaView)
! cp /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/figures/mesh/example_C.png .
Image("example_C.png")

**Fig.** The two layered velocity (Vp) model with modified mesh. The 15 km thick top layer has Vp = 2.8 km/s, while the 45 km thick bottom layer has Vp = 4.0 km/s. The discretization is the same for the top layer, with 4 elements in the 'Z' direction, while the bottom layer has about half the elements (~6 elements) in the 'Z' direction, as compared to the previous examples. The bottom layer also has half the elements (16 elements) along the 'X' and 'Y' directions as compared to that in the top layer (32 elements) of this example.

In [ ]:
# Run the solver (xspecfem3D)
%time ! mpiexec -np 4 ./bin/xspecfem3D

>__NOTE:__ solver runs should take ~5 minutes or less on a recent (as on 2022) laptop.

Note the wall time for the simulation.

In [ ]:
# Plot the output seismograms
X20_Z = np.genfromtxt("OUTPUT_FILES/DB.X20.BXZ.semd", dtype=None, names=("time","BXZ"))
X30_Z = np.genfromtxt("OUTPUT_FILES/DB.X30.BXZ.semd", dtype=None, names=("time","BXZ"))
X40_Z = np.genfromtxt("OUTPUT_FILES/DB.X40.BXZ.semd", dtype=None, names=("time","BXZ"))
X50_Z = np.genfromtxt("OUTPUT_FILES/DB.X50.BXZ.semd", dtype=None, names=("time","BXZ"))

t = X20_Z["time"]

plt.title("Seismograms (Z - component)")
plt.xlabel("---- time -->")
plt.ylabel("---- displacement -->")

plt.plot(t,X20_Z["BXZ"],label="X20")
plt.plot(t,X30_Z["BXZ"],label="X30")
plt.plot(t,X40_Z["BXZ"],label="X40")
plt.plot(t,X50_Z["BXZ"],label="X50")

plt.legend(title="Station")
plt.savefig("seis.png")

- By increasing the element size for the higher velocity layer, we are able to speed up the simulation greatly without any loss of accuracy in the results.  
- Selecting mesh element dimensions is governed by several factors -
    - frequencies of interest in the seismograms
    - velocity model
    - computational accuracy required
    - computational time available
- The quality of the mesh, as described in detail in [Komatitsch & Tromp 2002a](https://academic.oup.com/gji/article/149/2/390/727101), is reflected by -
    - number of grid points per wavelength
    - the stability condition

In [ ]:
# Remove the large sized database files
! rm -rf /home/scoped/work/day_1/specfem3d_workdir/OUTPUT_FILES/DATABASES_MPI

# Archive the output files, and the mesh files
! mv example_C.png OUTPUT_FILES/mesh_examples_C.png
! mv seis.png OUTPUT_FILES/.
! mv OUTPUT_FILES OUTPUT_FILES_example_C
! mv DATA/meshfem3D_files DATA/meshfem3D_files_example_C

-----------

**Congratulations! You have reached your destination.**